In [1]:
from os import listdir
from os.path import join, splitext, basename
import glob

import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import PIL
import numpy as np
import torch
import torchvision
import random
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

from imgaug import augmenters as iaa
import imgaug as ia

In [2]:
code2names = {
    0:"Bread",
    1:"Dairy_product",
    2:"Dessert",
    3:"Egg",
    4:"Fried_food",
    5:"Meat",
    6:"Noodles",
    7:"Rice",
    8:"Seafood",
    9:"Soup",
    10:"Vegetable_fruit"
}

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])

def load_img(filepath):
    img = Image.open(filepath)
    return img


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['figure.figsize'] = 15, 25
def show_dataset(dataset, n=6):
    img = np.vstack((np.hstack((np.asarray(dataset[i][0]) for _ in range(n))) for i in range(5)))
    plt.imshow(img)
    plt.axis('off')

In [4]:
class ImgAugTransform:
    def __init__(self):
        self.aug = iaa.Sequential([
            iaa.Scale((224, 224)),
            iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),
            iaa.Fliplr(0.5),
            iaa.Affine(rotate=(-20, 20), mode='symmetric'),
            iaa.Sometimes(0.25,
                      iaa.OneOf([iaa.Dropout(p=(0, 0.1)),
                                 iaa.CoarseDropout(0.1, size_percent=0.5)])),  # 對batch中的一部分圖片應用一部分Augmenters,剩下的圖片應用另外的Augmenters。
            iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)  # 即修改色調和飽和度
        ])
      
    def __call__(self, img):
        img = np.array(img)
        return self.aug.augment_image(img)

In [5]:
def input_transform():
    return transforms.Compose([
        ImgAugTransform(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

def not_train_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

In [6]:
class Food11Dataset(data.Dataset):
    def __init__(self, image_dir, input_transform=input_transform, is_train=False):
        super(Food11Dataset, self).__init__()
        self.path_pattern = image_dir + '/**/*.*'
        self.files_list = glob.glob(self.path_pattern, recursive=True)
        self.datapath = image_dir
        self.image_filenames = []
        self.classes_file_name = {}
        self.num_per_classes = {}
        self.class_name_list = []
        for file in self.files_list:
            if is_image_file(file):
                self.image_filenames.append(file)
                class_name = int(basename(file).split("_")[0])
                if class_name not in self.class_name_list:
                    self.class_name_list.append(class_name)
                if class_name in self.num_per_classes:
                    self.num_per_classes[class_name] += 1
                    self.classes_file_name[class_name].append(file)
                else:
                    self.num_per_classes[class_name] = 1
                    self.classes_file_name[class_name] = []
                    self.classes_file_name[class_name].append(file)
                    

        self.input_transform = input_transform
        self.is_train = is_train

    def __getitem__(self, index):
        # TODO [Lab 2-1] Try to embed third-party augmentation functions into pytorch flow
        input_file = self.image_filenames[index]
        input = load_img(input_file)
        if self.input_transform:
            input = self.input_transform()(input)
        label = basename(self.image_filenames[index])
        label = int(label.split("_")[0])
        return input, label

    def __len__(self):
        return len(self.image_filenames)

    def show_details(self):
        for key in sorted(self.num_per_classes.keys()):
            print("{:<8}|{:<20}|{:<12}".format(
                key,
                code2names[key],
                self.num_per_classes[key]
            ))
    
    ''' TODO [Lab 2-1]
    #please add a new function "augmentation(self, wts)"
    #it can change the number of data according to weight of each category
    #"weight" represents the ratio comparing with the amount of original set
    #if the weight > 100, we create new data by copying
    #if the weight < 100, we will delete the original data
    #[hint]you only need to edit the "self.image_filenames" 
ODOT [Lab 2-1]'''
    
    def augmentation(self):
        if self.is_train:
            pass

    def augmentation(self, wts): 
        outsample_num = {}
        if self.is_train:
            for w in range(len(wts)):
                outsample_num[w] = (int(self.num_per_classes[w] * wts[w]))
        
            for class_name in self.class_name_list:
                while self.num_per_classes[class_name] < outsample_num[class_name]:
                    random_file = random.sample(self.classes_file_name[class_name], 1)[0]
                    self.image_filenames.append(random_file)
                    self.num_per_classes[class_name] += 1
                    self.classes_file_name[class_name].append(random_file)
                while self.num_per_classes[class_name] > outsample_num[class_name]:
                    random_file = random.sample(self.classes_file_name[class_name], 1)[0]
                    self.image_filenames.remove(random_file)
                    self.num_per_classes[class_name] -= 1
                    self.classes_file_name[class_name].remove(random_file)

                    
                       

In [7]:
def WRSampler(dataset, wts):
    class_name_list = dataset.class_name_list
    num_per_classes = dataset.num_per_classes
    
    each_data_wts = []
    for class_name in class_name_list:
        class_item_num = num_per_classes[class_name]
        for i in range(class_item_num):
            each_data_wts.append(wts[class_name])
    
    sampler = torch.utils.data.sampler.WeightedRandomSampler(each_data_wts, len(each_data_wts), replacement=True)
    
    return sampler

In [8]:
def data_loading(loader, dataset):

    num_per_classes = {}
    for batch_idx, (data, label) in enumerate(loader):
        for l in label:
            if l.item() in num_per_classes:
                num_per_classes[l.item()] += 1
            else:
                num_per_classes[l.item()] = 1

    print("----------------------------------------------------------------------------------")
    print("Dataset - ", dataset.datapath)
    print("{:<20}|{:<15}|{:<15}".format("class_name", "bf. loading", "af. loading"))
    for key in sorted(num_per_classes.keys()):
        print("{:<20}|{:<15}|{:<15}".format(
            code2names[key],
            dataset.num_per_classes[key],
            num_per_classes[key]
        ))


In [9]:
def main():
    train_datapath = "./training"
    valid_datapath = "./validation"
    test_datapath = "./evaluation"

    train_dataset = Food11Dataset(train_datapath, input_transform=input_transform, is_train=True)
    valid_dataset = Food11Dataset(valid_datapath, input_transform=not_train_transform, is_train=False)
    test_dataset = Food11Dataset(test_datapath, input_transform=not_train_transform, is_train=False)

    ''' For [Lab 2-1] debugging
    train_dataset.augmentation()
    '''
    wts = [ 1.5, 3.5, 1, 1.52, 1.76, 1.13, 3.4, 5.35, 1.75, 1, 2.1 ]
    
    train_dataset.augmentation(wts)
    #show_dataset(train_dataset)
    
    print("----------------------------------------------------------------------------------")
    print("Dataset bf. loading - ", train_datapath)
    print(train_dataset.show_details())

    print("----------------------------------------------------------------------------------")
    print("Dataset bf. loading - ", valid_datapath)
    print(valid_dataset.show_details())

    print("----------------------------------------------------------------------------------")
    print("Dataset bf. loading - ", test_datapath)
    print(test_dataset.show_details())
   
    #sampler = WRSampler(train_dataset, wts)
    #sampler = torch.utils.data.sampler.RandomSampler(data_source=train_dataset, replacement=True, num_samples=len(train_dataset))

    train_loader = DataLoader(dataset=train_dataset, num_workers=4, batch_size=8)#, sampler = sampler)
    valid_loader = DataLoader(dataset=valid_dataset, num_workers=4, batch_size=8, shuffle=False)
    test_loader = DataLoader(dataset=test_dataset, num_workers=4, batch_size=8, shuffle=False)

    data_loading(train_loader, train_dataset)
    data_loading(valid_loader, valid_dataset)
    data_loading(test_loader, test_dataset)



In [10]:
if __name__ == '__main__':
    #main()
    pass

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_datapath = "./training"
valid_datapath = "./validation"
test_datapath = "./evaluation"

wts = [ 1.5, 3.5, 1, 1.52, 1.76, 1.13, 3.4, 5.35, 1.75, 1, 2.1 ]

train_dataset = Food11Dataset(train_datapath, input_transform=input_transform, is_train=True)
valid_dataset = Food11Dataset(valid_datapath, input_transform=not_train_transform, is_train=False)
test_dataset = Food11Dataset(test_datapath, input_transform=not_train_transform, is_train=False)
#train_dataset.augmentation(wts)

sampler = WRSampler(train_dataset, wts)

train_loader = DataLoader(dataset=train_dataset, num_workers=4, batch_size=32, sampler = sampler)
valid_loader = DataLoader(dataset=valid_dataset, num_workers=4, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, num_workers=4, batch_size=8, shuffle=False)

net = models.resnet18(pretrained=False)
net.fc = nn.Sequential(nn.Linear(512,256),nn.LeakyReLU(),nn.Linear(256,128),nn.LeakyReLU(),nn.Linear(128,11))
net = net.to(device) 

learning_rate = 0.0001
criterion = nn.CrossEntropyLoss() #定義損失函數
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, betas=[0.9, 0.999]) #定優化函數

net.train()
num_epoch = 200
run_epoch = 0
t_loss = []
v_loss = []
training_accuracy = []
validation_accuracy = []
total = 0

for epoch in range(200):  # loop over the dataset multiple times
    train_loss = 0.0
    validation_loss = 0.0
    correct_train = 0
    correct_validation = 0
    train_num = 0
    val_num = 0
    cls = []

    for i in range(11):
        cls.append(0)
    
    ########################
    # train the model      #
    ########################
    
    net.train()
    for i, (inputs, labels) in enumerate(train_loader, 0):
        
        #change the type into cuda tensor 
        inputs = inputs.to(device) 
        labels = labels.to(device) 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # select the class with highest probability
        _, pred = outputs.max(1)
        # if the model predicts the same results as the true
        # label, then the correct counter will plus 1
        correct_train += pred.eq(labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        train_loss += loss.item()
        train_num += 1
   

        
    ########################
    # validate the model   #
    ########################
    
    net.eval()
    for i, (inputs, labels) in enumerate(valid_loader, 0):
        # move tensors to GPU if CUDA is available
        inputs = inputs.to(device) 
        labels = labels.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = net(inputs)
        _, pred = outputs.max(1)
        correct_validation += pred.eq(labels).sum().item()
        # calculate the batch loss
        loss = criterion(outputs, labels)
        # update average validation loss 
        validation_loss += loss.item()
        val_num += 1
        
        
    if epoch % 1 == 0:    # print every 200 mini-batches
        print('[%d, %5d] train_loss: %.3f' % (epoch + 1, num_epoch, train_loss / train_num))
        print('[%d, %5d] validation_loss: %.3f' % (epoch + 1, num_epoch, validation_loss / val_num))
        print('%d epoch, training accuracy: %.4f' % (epoch + 1, correct_train / len(train_dataset)))
        print('%d epoch, validation accuracy: %.4f' % (epoch + 1, correct_validation / len(valid_dataset)))
        print('-----------------------------------------')
        
        t_loss.append(train_loss / train_num)
        training_accuracy.append(correct_train / len(train_dataset))
        validation_accuracy.append(correct_validation / len(valid_dataset))
        running_loss = 0.0
        validation_loss = 0.0
        train_num = 0
        val_num = 0
        correct_train = 0
        correct_validation = 0
        total = 0
        run_epoch += 1
        
        torch.save(net.state_dict(), './resnet18_no_augmentation/' + str(epoch) + '.pth')
    
print('Finished Training')

/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


[1,   200] train_loss: 2.103
[1,   200] validation_loss: 2.188
1 epoch, training accuracy: 0.2605
1 epoch, validation accuracy: 0.2312
-----------------------------------------


/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


[2,   200] train_loss: 1.819
[2,   200] validation_loss: 1.766
2 epoch, training accuracy: 0.3689
2 epoch, validation accuracy: 0.3840
-----------------------------------------


/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


...

/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


[108,   200] train_loss: 0.096
[108,   200] validation_loss: 1.611
108 epoch, training accuracy: 0.9674
108 epoch, validation accuracy: 0.6831
-----------------------------------------


/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


[109,   200] train_loss: 0.087
[109,   200] validation_loss: 1.468
109 epoch, training accuracy: 0.9699
109 epoch, validation accuracy: 0.7108
-----------------------------------------


/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)
/home/s1041549/anaconda3/envs/dlsystem/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `Scale()` is deprecated. Use `Resize` instead. Resize has the exactly same interface as Scale.
  warn_deprecated(msg, stacklevel=3)


In [ ]:
def reload_net():
    trainednet = models.resnet18()  
    #trainednet.fc = torch.nn.Linear(1000, 11)  
    trainednet.fc = nn.Sequential(nn.Linear(512,256),nn.LeakyReLU(),nn.Linear(256,128),nn.LeakyReLU(),nn.Linear(128,11))
    trainednet.load_state_dict(torch.load('./resnet18_no_augmentation/38.pth'))
    return trainednet

In [ ]:
pretrain_net = reload_net().to(device)
test_loss = 0.0
correct_test = 0
test_num = 0
cls = np.zeros(11)
correct_top3 = 0

In [ ]:
pretrain_net.eval()
for i, (inputs, labels) in enumerate(test_loader, 0):
    # move tensors to GPU if CUDA is available
    inputs = inputs.to(device) 
    labels = labels.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    outputs = pretrain_net(inputs)
    _, pred = outputs.max(1)
    correct_test += pred.eq(labels).sum().item()
    _, top3 = outputs.topk(3)
    correct_top3 += top3.eq(labels.view(-1,1).expand_as(top3)).sum().item()
    
    cls[0] += (pred.eq(0) * pred.eq(labels)).sum().item()
    cls[1] += (pred.eq(1) * pred.eq(labels)).sum().item()
    cls[2] += (pred.eq(2) * pred.eq(labels)).sum().item()
    cls[3] += (pred.eq(3) * pred.eq(labels)).sum().item()
    cls[4] += (pred.eq(4) * pred.eq(labels)).sum().item()
    cls[5] += (pred.eq(5) * pred.eq(labels)).sum().item()
    cls[6] += (pred.eq(6) * pred.eq(labels)).sum().item()
    cls[7] += (pred.eq(7) * pred.eq(labels)).sum().item()
    cls[8] += (pred.eq(8) * pred.eq(labels)).sum().item()
    cls[9] += (pred.eq(9) * pred.eq(labels)).sum().item()
    cls[10] += (pred.eq(10) * pred.eq(labels)).sum().item()

In [ ]:
print('Test set: Top 1 Accuracy: %d/3347 (%.2f%%), Top 3 Accuracy: %d/3347 (%.2f%%)' 
      % (correct_test, correct_test / len(test_dataset)*100, correct_top3, correct_top3/ len(test_dataset)*100))
print('%-20s : %d/%d    %10f%%' % (code2names[0], cls[0], 368, cls[0]/368*100))
print('%-20s : %d/%d    %10f%%' % (code2names[1], cls[1], 148, cls[1]/148*100))
print('%-20s : %d/%d    %10f%%' % (code2names[2], cls[2], 500, cls[2]/500*100))
print('%-20s : %d/%d    %10f%%' % (code2names[3], cls[3], 335, cls[3]/335*100))
print('%-20s : %d/%d    %10f%%' % (code2names[4], cls[4], 287, cls[4]/287*100))
print('%-20s : %d/%d    %10f%%' % (code2names[5], cls[5], 432, cls[5]/432*100))
print('%-20s : %d/%d    %10f%%' % (code2names[6], cls[6], 147, cls[6]/147*100))
print('%-20s : %d/%d    %10f%%' % (code2names[7], cls[7], 96, cls[7]/96*100))
print('%-20s : %d/%d    %10f%%' % (code2names[8], cls[8], 303, cls[8]/303*100))
print('%-20s : %d/%d    %10f%%' % (code2names[9], cls[9], 500, cls[9]/500*100))
print('%-20s : %d/%d    %10f%%' % (code2names[10], cls[10], 231, cls[10]/231*100))

avg = []
avg.append(cls[0]/368*100)
avg.append(cls[1]/148*100)
avg.append(cls[2]/500*100)
avg.append(cls[3]/335*100)
avg.append(cls[4]/287*100)
avg.append(cls[5]/432*100)
avg.append(cls[6]/147*100)
avg.append(cls[7]/96*100)
avg.append(cls[8]/303*100)
avg.append(cls[9]/500*100)
avg.append(cls[10]/231*100)
print('Average per case accuracy: %10f%%' % (sum(avg)/len(avg)))
print('-----------------------------------------')